In [5]:
# Дескриптор атрибута для целочисленного атрибута с проверкой типа
class Integer:
    def __init__(self, name):
        self.name = name

    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            return instance.__dict__[self.name]

    def __set__(self, instance, value):
        if not isinstance(value, int):
            raise TypeError('Expected an int')
        instance.__dict__[self.name] = value

    def __delete__(self, instance):
        del instance.__dict__[self.name]
        

class Point:
    x = Integer('x')
    y = Integer('y')
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
p = Point(2, 3)
print(p.x) # Вызывает Point.x.__get__(p,Point)
p.y = 5 # Вызывает Point.y.__set__(p, 5)
print(p.y)

2
5


In [6]:
class lazyproperty:
    def __init__(self, func):
        self.func = func
    
    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            value = self.func(instance)
            setattr(instance, self.func.__name__, value)
            return value
        

class Circle:
    def __init__(self, radius):
        self.radius = radius
    
    @lazyproperty
    def area(self):
        print('Computing area')
        return math.pi * self.radius ** 2
    
    @lazyproperty
    def perimeter(self):
        print('Computing perimeter')
        return 2 * math.pi * self.radius
    

c = Circle(4.0)
print(c.radius)
print(c.area)
print(c.area)
print(c.perimeter)
print(c.perimeter)

# Получение переменных экземпляра
print(vars(c))

4.0
Computing area
50.26548245743669
50.26548245743669
Computing perimeter
25.132741228718345
25.132741228718345
{'radius': 4.0, 'area': 50.26548245743669, 'perimeter': 25.132741228718345}


In [7]:
class Structure:
    # Переменная класса, которая определяет ожидаемые поля
    _fields= []
    def __init__(self, *args):
        if len(args) != len(self._fields):
            raise TypeError(f'Expected {len(self._fields)} arguments')
        
        # Устанавливает аргументы
        for name, value in zip(self._fields, args):
            setattr(self, name, value)
            
    
# Пример определения класса
class Stock(Structure):
    _fields = ['name', 'shares', 'price']
    
class Point(Structure):
    _fields = ['x','y']
    
class Circle(Structure):
    _fields = ['radius']
    def area(self):
        return math.pi * self.radius ** 2
    
s = Stock('ACME', 50, 91.1)
p = Point(2, 3)
c = Circle(4.5)

print(s)
print(p)
print(c.area())

63.61725123519331


In [8]:
import io
from abc import ABCMeta, abstractmethod

class IStream(metaclass=ABCMeta):
    @abstractmethod
    def read(self, maxbytes=-1):
        pass
    
    @abstractmethod
    def write(self, data):
        pass
    
def serialize(obj, stream):
    if not isinstance(stream, IStream):
        raise TypeError('Expected an IStream')
    
# Регистрируем встроенные классы ввода-вывода
# в качестве поддерживающих для нашего интерфейса
IStream.register(io.IOBase)

# Открыть обычный файл и провести проверку типа
with open('foo.txt') as f:
    print(isinstance(f, IStream))

True


In [9]:
# Базовый класс. Использует дескриптор для установки значения
class Descriptor:
    def __init__(self, name=None, **opts):
        self.name = name
        for key, value in opts.items():
            setattr(self, key, value)
            
    def __set__(self, instance, value):
        instance.__dict__[self.name] = value
        
# Дескриптор для принудительного определения типов
class Typed(Descriptor):
    expected_type = type(None)
    
    def __set__(self, instance, value):
        if not isinstance(value, self.expected_type):
            raise TypeError('expected ' + str(self.expected_type))
        super().__set__(instance, value)
        
# Дескриптор для принудительного определения значений
class Unsigned(Descriptor):
    def __set__(self, instance, value):
        if value < 0:
            raise ValueError('Expected >= 0')
        super().__set__(instance, value)
        
class MaxSized(Descriptor):
    def __init__(self, name=None, **opts):
        if 'size' not in opts:
            raise TypeError('missing size option')
        super().__init__(name, **opts)
        
    def __set__(self, instance, value):
        if len(value) >= self.size:
            raise ValueError('size must be < ' + str(self.size))
        super().__set__(instance, value)
        

class Integer(Typed):
    expected_type = int

class UnsignedInteger(Integer, Unsigned):
    pass

class Float(Typed):
    expected_type = float

class UnsignedFloat(Float, Unsigned):
    pass

class String(Typed):
    expected_type = str

class SizedString(String, MaxSized):
    pass


class Stock:
    # Определяем ограничения
    name = SizedString('name', size=8)
    shares = UnsignedInteger('shares')
    price = UnsignedFloat('price')
    
    def __init__(self, name, shares, price):
        self.name = name
        self.shares = shares
        self.price = price
        
        
s = Stock('ACME', 50, 91.1)
print(s.name)

s.shares = 10 
print(s.shares)       
        

ACME
10


In [10]:
import bisect
from collections.abc import Sequence

class SortedItems(Sequence):
    def __init__(self, initial=None):
        self._items = sorted(initial) if initial is not None else []

    # Требуемые методы последовательности
    def __getitem__(self, index):
        return self._items[index]

    def __len__(self):
        return len(self._items)

    # Метод для добавления элемента в правильное место
    def add(self, item):
        bisect.insort(self._items, item)
        

items = SortedItems([5, 1, 3])
print(list(items))
items.add(2)
print(list(items))

[1, 3, 5]
[1, 2, 3, 5]


In [11]:
from collections.abc import MutableSequence

class Items(MutableSequence):
    def __init__(self, initial=None):
        self._items = list(initial) if initial is not None else []

    # Обязательные методы последовательности
    def __getitem__(self, index):
        print('Getting:', index)
        return self._items[index]
    
    def __setitem__(self, index, value):
        print('Setting:', index, value)
        self._items[index] = value
    
    def __delitem__(self, index):
        print('Deleting:', index)
        del self._items[index]
    
    def insert(self, index, value):
        print('Inserting:', index, value)
        self._items.insert(index, value)
    
    def __len__(self):
        print('Len')
        return len(self._items)
    
a = Items([1, 2, 3])
print(len(a))
a.append(4)
a.count(2)
a.remove(3)

Len
3
Len
Inserting: 3 4
Getting: 0
Getting: 1
Getting: 2
Getting: 3
Getting: 4
Getting: 0
Getting: 1
Getting: 2
Deleting: 2


In [12]:
class A:
    def spam(self, x):
        print("Spam:", x)
    
    def foo(self):
        pass

class B:
    def __init__(self):
        self._a = A()
    
    def bar(self):
        print("B.bar()")
    
    # Показывает все методы, определенные на классе A
    def __getattr__(self, name):
        return getattr(self._a, name)
    
b = B()
b.bar() # Вызывает B.bar() (существует на B)   
b.spam(42) # Вызывает B.__getattr__('spam') и делегирует A.spam   

B.bar()
Spam: 42


In [13]:
class Proxy:
    '''
    Класс-прокси, который оборачивается вокруг другого объекта,
    но показывает его публичные атрибуты
    '''
    def __init__(self, obj):
        self._obj = obj

    def __getattr__(self, name):
        '''Делегирует поиск атрибутов внутреннему obj'''
        print('getattr:', name)
        return getattr(self._obj, name)

    def __setattr__(self, name, value):
        '''Делегирует присвоение атрибутов'''
        if name.startswith('_'):
            super().__setattr__(name, value)
        else:
            print('setattr:', name, value)
            setattr(self._obj, name, value)

    def __delattr__(self, name):
        '''Делегирует удаление атрибутов'''
        if name.startswith('_'):
            super().__delattr__(name)
        else:
            print('delattr:', name)
            delattr(self._obj, name)
            

class Spam:
    def __init__(self, x):
        self.x = x
    
    def bar(self, y):
        print('Spam.bar:', self.x, y)
        
# Создаем экземпляр
s = Spam(2)
# Создаем прокси вокруг него
p = Proxy(s)

# Access the proxy
print(p.x) 
p.bar(3) 
p.x = 37     

getattr: x
2
getattr: bar
Spam.bar: 2 3
setattr: x 37


In [14]:
class Connection:
    def __init__(self):
        self.new_state(ClosedConnectionState)
    
    def new_state(self, newstate):
        self._state = newstate
        
    # Делегирует классу состояния
    def read(self):
        return self._state.read(self)
    
    def write(self, data):
        return self._state.write(self, data)
    
    def open(self):
        return self._state.open(self)
    
    def close(self):
        return self._state.close(self)

# Базовый класс состояния соединения
class ConnectionState:
    @staticmethod
    def read(conn):
        raise NotImplementedError()
    
    @staticmethod
    def write(conn, data):
        raise NotImplementedError()
    
    @staticmethod
    def open(conn):
        raise NotImplementedError()
    
    @staticmethod
    def close(conn):
        raise NotImplementedError()
    
    
# Реализация различных состояний
class ClosedConnectionState(ConnectionState):
    @staticmethod
    def read(conn):
        raise RuntimeError('Not open')

    @staticmethod
    def write(conn, data):
        raise RuntimeError('Not open')

    @staticmethod
    def open(conn):
        conn.new_state(OpenConnectionState)

    @staticmethod
    def close(conn):
        raise RuntimeError('Already closed')
    
    
class OpenConnectionState(ConnectionState):
    @staticmethod
    def read(conn):
        print('reading')
    
    @staticmethod
    def write(conn, data):
        print('writing')
    
    @staticmethod
    def open(conn):
        raise RuntimeError('Already open')
    
    @staticmethod
    def close(conn):
        conn.new_state(ClosedConnectionState)
        

c = Connection()
print(c._state)
c.open()
print(c._state)
c.read()
c.write('hello')    

<class '__main__.ClosedConnectionState'>
<class '__main__.OpenConnectionState'>
reading
writing


In [15]:
import math
import operator

class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __repr__(self):
        return f'Point({self.x!r:},{self.y!r:})'
    
    def distance(self, x, y):
        return math.hypot(self.x - x, self.y - y)
    
p = Point(2, 3)
d = getattr(p, 'distance')(0, 0) # Вызов p.distance(0, 0)
print(d)
print(operator.methodcaller('distance', 0, 0)(p))

3.605551275463989
3.605551275463989


In [16]:
points = [
    Point(1, 2),
    Point(3, 0),
    Point(10, -3),
    Point(-5, -7),
    Point(-1, 8),
    Point(3, 2)
]

# Сортируем по расстоянию от (0, 0)
points.sort(key=operator.methodcaller('distance', 0, 0))
print(points)

[Point(1,2), Point(3,0), Point(3,2), Point(-1,8), Point(-5,-7), Point(10,-3)]


In [17]:
import weakref

class Node:
    def __init__(self, value):
        self.value = value
        self._parent = None
        self.children = []

    def __repr__(self):
        return 'Node({!r:})'.format(self.value)

    # Свойство, которое управляет родителем с помощью слабой ссылки
    @property
    def parent(self):
        return self._parent if self._parent is None else self._parent()

    @parent.setter
    def parent(self, node):
        self._parent = weakref.ref(node)

    def add_child(self, child):
        self.children.append(child)
        child.parent = self
        

root = Node('parent')
c1 = Node('child')
root.add_child(c1)
print(c1.parent)
del root
print(c1.parent)

Node('parent')
None


In [18]:
# Класс, созданный, чтобы проиллюстрировать то, что будет при удалении
class Data:
    def __del__(self):
        print('Data.__del__')

# Класс-узел, в котором есть цикл
class Node:
    def __init__(self):
        self.data = Data()
        self.parent = None
        self.children = []
        
    def add_child(self, child):
        self.children.append(child)
        child.parent = self
        
a = Data()
del a # Сразу же удаляется
a = Node()
del a # Сразу же удаляется
a = Node()
a.add_child(Node())
del a # Не удаляется (и сообщение не выводится)

Data.__del__
Data.__del__


In [19]:
from functools import total_ordering

class Room:
    def __init__(self, name, length, width):
        self.name = name
        self.length = length
        self.width = width
        self.square_feet = self.length * self.width

@total_ordering
class House:
    def __init__(self, name, style):
        self.name = name
        self.style = style
        self.rooms = list()

    @property
    def living_space_footage(self):
        return sum(r.square_feet for r in self.rooms)

    def add_room(self, room):
        self.rooms.append(room)

    def __str__(self):
        return f'{self.name}: {self.living_space_footage} square foot {self.style}'

    def __eq__(self, other):
        return self.living_space_footage == other.living_space_footage
    
    def __lt__(self, other):
        return self.living_space_footage < other.living_space_footage
    
    
# Построим несколько домов и добавим в них комнаты
h1 = House('h1', 'Cape')
h1.add_room(Room('Master Bedroom', 14, 21))
h1.add_room(Room('Living Room', 18, 20))
h1.add_room(Room('Kitchen', 12, 16))
h1.add_room(Room('Office', 12, 12))

h2 = House('h2', 'Ranch')
h2.add_room(Room('Master Bedroom', 14, 21))
h2.add_room(Room('Living Room', 18, 20))
h2.add_room(Room('Kitchen', 12, 16))

h3 = House('h3', 'Split')
h3.add_room(Room('Master Bedroom', 14, 21))
h3.add_room(Room('Living Room', 18, 20))
h3.add_room(Room('Office', 12, 16))
h3.add_room(Room('Kitchen', 15, 17))

houses = [h1, h2, h3]

print('Is h1 bigger than h2?', h1 > h2)
print('Is h2 smaller than h3?', h2 < h3)
print('Is h2 greater than or equal to h1?', h2 >= h1)
print('Which one is biggest?', max(houses))
print('Which is smallest?', min(houses))

Data.__del__
Data.__del__
Is h1 bigger than h2? True
Is h2 smaller than h3? True
Is h2 greater than or equal to h1? False
Which one is biggest? h3: 1101 square foot Split
Which is smallest? h2: 846 square foot Ranch


In [20]:
import logging

a = logging.getLogger('foo')
b = logging.getLogger('bar')
print(a is b)
c = logging.getLogger('foo')
print(a is c)

# Опрашиваемый класс
class Spam:
    def __init__(self, name):
        self.name = name
        
# Поддержка кеширования
_spam_cache = weakref.WeakValueDictionary()

def get_spam(name):
    if name not in _spam_cache:
        s = Spam(name)
        _spam_cache[name] = s
    else:
        s = _spam_cache[name]
    return s

a = get_spam('foo')
b = get_spam('bar')
print(a is b)
c = get_spam('foo')
print(a is c)

False
True
False
True


In [21]:
class CachedSpamManager:
    def __init__(self):
        self._cache = weakref.WeakValueDictionary()
    
    def get_spam(self, name):
        if name not in self._cache:
            s = Spam(name)
            self._cache[name] = s
        else:
            s = self._cache[name]
        return s
    
    def clear(self):
        self._cache.clear()

class Spam:
    manager = CachedSpamManager()
    def __init__(self, name):
        self.name = name
    
    def get_spam(name):
        return Spam.manager.get_spam(name)
    
a = Spam('foo')
b = Spam('foo')
a is b

False

In [22]:
import weakref

class Cached(type):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.__cache = weakref.WeakValueDictionary()
    
    def __call__(self, *args):
        if args in self.__cache:
            return self.__cache[args]
        else:
            obj = super().__call__(*args)
            self.__cache[args] = obj
            return obj

# Пример
class Spam(metaclass=Cached):
    def __init__(self, name):
        print(f'Creating Spam({name!r})')
        self.name = name
        

a = Spam('Guido')
b = Spam('Diana')
c = Spam('Guido') # Закеширован
print(a is b)
print(a is c) # Возвращается закешированное значение

Creating Spam('Guido')
Creating Spam('Diana')
False
True


In [23]:
from collections import OrderedDict

# Набор дескрипторов для различных типов
class Typed:
    _expected_type = type(None)

    def __init__(self, name=None):
        self._name = name

    def __set__(self, instance, value):
        if not isinstance(value, self._expected_type):
            raise TypeError('Expected ' + str(self._expected_type))
        instance.__dict__[self._name] = value

class Integer(Typed):
    _expected_type = int

class Float(Typed):
    _expected_type = float

class String(Typed):
    _expected_type = str

# Метакласс, который использует OrderedDict для тела класса
class OrderedMeta(type):
    def __new__(cls, clsname, bases, clsdict):
        d = dict(clsdict)
        order = []
        for name, value in clsdict.items():
            if isinstance(value, Typed):
                value._name = name
                order.append(name)
        d['_order'] = order
        return type.__new__(cls, clsname, bases, d)
    
    @classmethod
    def __prepare__(cls, clsname, bases):
        return OrderedDict()
    
    
class Structure(metaclass=OrderedMeta):
    def as_csv(self):
        return ', '.join(str(getattr(self,name)) for name in self._order)

# Пример использования
class Stock(Structure):
    name = String()
    shares = Integer()
    price = Float()
    
    def __init__(self, name, shares, price):
        self.name = name
        self.shares = shares
        self.price = price
        

s = Stock('GOOG', 100, 490.1)
print(s.name)
print(s.as_csv())    

GOOG
GOOG, 100, 490.1


In [24]:
# Пример ручного создания класса из частей

# Методы
def __init__(self, name, shares, price):
    self.name = name
    self.shares = shares
    self.price = price

def cost(self):
    return self.shares * self.price

cls_dict = {'__init__' : __init__, 'cost' : cost}

# Создание класса
import types

Stock = types.new_class('Stock', (), {}, lambda ns: ns.update(cls_dict))
Stock.__module__ = __name__

s = Stock('ACME', 50, 91.1)
print(s)
print(s.cost())

4555.0


In [27]:
import abc

Stock = types.new_class('Stock', (), {'metaclass': abc.ABCMeta}, lambda ns: ns.update(cls_dict))
Stock.__module__ = __name__

print(Stock)
print(type(Stock))

<class '__main__.Stock'>
<class 'abc.ABCMeta'>


In [31]:
import operator
import types
import sys

def named_tuple(classname, fieldnames):
    # Наполняем словарь акцессоров свойств полей
    cls_dict = {name: property(operator.itemgetter(n)) for n, name in enumerate(fieldnames)}

    # Создаем функцию __new__ и добавляем ее в словарь класса
    def __new__(cls, *args):
        if len(args) != len(fieldnames):
            raise TypeError(f'Expected {len(fieldnames)} arguments')
        return tuple.__new__(cls, args)
    
    cls_dict['__new__'] = __new__
    
    # Создаем класс
    cls = types.new_class(classname, (tuple,), {}, lambda ns: ns.update(cls_dict))
    
    # Устанавливаем модуль класса на модуль вызывающего
    cls.__module__ = sys._getframe(1).f_globals['__name__']
    return cls


Point = named_tuple('Point', ['x', 'y'])
print(Point)

p = Point(4, 5)
print(len(p))
print(p.x)
print(p.y)

<class '__main__.Point'>
2
4
5


In [34]:
import operator

class StructTupleMeta(type):
    def __init__(cls, *args, **kwargs):
        super().__init__(*args, **kwargs)
        for n, name in enumerate(cls._fields):
            setattr(cls, name, property(operator.itemgetter(n)))

class StructTuple(tuple, metaclass=StructTupleMeta):
    _fields = []
    def __new__(cls, *args):
        if len(args) != len(cls._fields):
            raise ValueError(f'{len(cls._fields)} arguments required')
        return super().__new__(cls,args)
    

class Stock(StructTuple):
    _fields = ['name', 'shares', 'price']
    
class Point(StructTuple):
    _fields = ['x', 'y']
    

s = Stock('ACME', 50, 91.1)
print(s)
print(s[0])
print(s.name)
print(s.shares * s.price)

('ACME', 50, 91.1)
ACME
ACME
4555.0


In [11]:
import inspect
import types

class MultiMethod:
    '''Представляет один мультиметод.'''
    def __init__(self, name):
        self._methods = {}
        self.__name__ = name
        
    def register(self, meth):
        '''Регистрирует новый метод как мультиметод'''
        sig = inspect.signature(meth)
        # Создание сигнатуры типа из аннотаций методов
        types = []
        for name, parm in sig.parameters.items():
            if name == 'self':
                continue
            if parm.annotation is inspect.Parameter.empty:
                raise TypeError(f'Argument {name} must be annotated with a type')
            if not isinstance(parm.annotation, type):
                raise TypeError(f'Argument {name} annotation must be a type')
            if parm.default is not inspect.Parameter.empty:
                self._methods[tuple(types)] = meth
            
            types.append(parm.annotation)
        self._methods[tuple(types)] = meth
        
    def __call__(self, *args):
        '''Вызов метода базируется на сигнатуре типа аргументов'''
        types = tuple(type(arg) for arg in args[1:])
        meth = self._methods.get(types, None)
        if meth:
            return meth(*args)
        else:
            raise TypeError(f'No matching method for types {types}')
        
    def __get__(self, instance, cls):
        '''Метод дескриптора, необходимый для работы вызовов в классе'''
        if instance is not None:
            return types.MethodType(self, instance)
        else:
            return self
        

class MultiDict(dict):
    '''Специальный словарь для создания мультиметодов в метаклассе'''
    def __setitem__(self, key, value):
        if key in self:
            # Если ключ уже существует, он должен быть мультиметодом
            # или вызываемым объектом
            current_value = self[key]
            if isinstance(current_value, MultiMethod):
                current_value.register(value)
            else:
                mvalue = MultiMethod(key)
                mvalue.register(current_value)
                mvalue.register(value)
                super().__setitem__(key, mvalue)
        else:
            super().__setitem__(key, value)
            

class MultipleMeta(type):
    '''Метакласс, который позволяет множественную диспетчеризацию методов'''
    def __new__(cls, clsname, bases, clsdict):
        return type.__new__(cls, clsname, bases, dict(clsdict))
    
    @classmethod
    def __prepare__(cls, clsname, bases):
        return MultiDict()
    

class Spam(metaclass=MultipleMeta):
    def bar(self, x:int, y:int):
        print('Bar 1:', x, y)
    
    def bar(self, s:str, n:int = 0):
        print('Bar 2:', s, n)
        

s = Spam()
s.bar(2, 3)
s.bar('hello', 15)
print()

# Пример: перегруженный __init__
import time

class Date(metaclass=MultipleMeta):
    def __init__(self, year: int, month:int, day:int):
        self.year = year
        self.month = month
        self.day = day
        
    def __init__(self):
        t = time.localtime()
        self.__init__(t.tm_year, t.tm_mon, t.tm_mday)
        

# Перегруженный __init__
d = Date(2012, 12, 21)
print(d.year)
print(d.month)
print(d.day)
print()

# Получить сегодняшнюю дату
e = Date()
print(e.year)
print(e.month)
print(e.day)   

Bar 1: 2 3
Bar 2: hello 15

2012
12
21

2025
11
6


In [13]:
class A:
    pass

class B(A):
    pass

class C:
    pass


class Spam(metaclass=MultipleMeta):
    def foo(self, x:A):
        print('Foo 1:', x)
    
    def foo(self, x:C):
        print('Foo 2:', x)
        

s = Spam()
a = A()
s.foo(a)
c = C()
s.foo(c)


Foo 1: <__main__.A object at 0x000001EBEEE2F5E0>
Foo 2: <__main__.C object at 0x000001EBEEE6E860>


In [16]:
import types

class multimethod:
    def __init__(self, func):
        self._methods = {}
        self.__name__ = func.__name__
        self._default = func
        
    def match(self, *types):
        def register(func):
            ndefaults = len(func.__defaults__) if func.__defaults__ else 0
            for n in range(ndefaults + 1):
                self._methods[types[:len(types) - n]] = func
            return self 
        return register
    
    def __call__(self, *args):
        types = tuple(type(arg) for arg in args[1:])
        meth = self._methods.get(types, None)
        if meth:
            return meth(*args)
        else:
            return self._default(*args)
        
    def __get__(self, instance, cls):
        if instance is not None:
            return types.MethodType(self, instance)
        else:
            return self
        
        
class Spam:
    @multimethod
    def bar(self, *args):
        # Если нет совпадений, вызывается дефолтный метод
        raise TypeError('No matching method for bar')
    
    @bar.match(int, int)
    def bar(self, x, y):
        print('Bar 1:', x, y)
    
    @bar.match(str, int)
    def bar(self, s, n = 0):
        print('Bar 2:', s, n)
        

s = Spam()
s.bar(2, 3)
s.bar('hello', 15)     

Bar 1: 2 3
Bar 2: hello 15


In [17]:
import ast

ex = ast.parse('2 + 3*4 + x', mode='eval')
print(ex)
print(ast.dump(ex))
print()
top = ast.parse('for i in range(10): print(i)', mode='exec')
print(top)
print(ast.dump(top))

Expression(body=BinOp(left=BinOp(left=Constant(value=2), op=Add(), right=BinOp(left=Constant(value=3), op=Mult(), right=Constant(value=4))), op=Add(), right=Name(id='x', ctx=Load())))

Module(body=[For(target=Name(id='i', ctx=Store()), iter=Call(func=Name(id='range', ctx=Load()), args=[Constant(value=10)], keywords=[]), body=[Expr(value=Call(func=Name(id='print', ctx=Load()), args=[Name(id='i', ctx=Load())], keywords=[]))], orelse=[])], type_ignores=[])


In [23]:
import ast

class CodeAnalyzer(ast.NodeVisitor):
    def __init__(self):
        self.loaded = set()
        self.stored = set()
        self.deleted = set()
        
    def visit_Name(self, node):
        if isinstance(node.ctx, ast.Load):
            self.loaded.add(node.id)
        elif isinstance(node.ctx, ast.Store):
            self.stored.add(node.id)
        elif isinstance(node.ctx, ast.Del):
            self.deleted.add(node.id)
            
# Пример использования
code = '''
for i in range(3):
    print(i)
del i
'''

# Парсим в AST
top = ast.parse(code, mode='exec')

# Скармливаем AST анализатору использования имен
c = CodeAnalyzer()
c.visit(top)
print('Loaded:', c.loaded)
print('Stored:', c.stored)
print('Deleted:', c.deleted)

exec(compile(top,'<stdin>', 'exec'))

Loaded: {'range', 'print', 'i'}
Stored: {'i'}
Deleted: {'i'}
0
1
2


In [3]:
'''Перегрузка операторов'''
class ReloadClass:
    def __init__(self):
        self.x = 0
        self.a = [1,2,3]
        
    def __eq__(self, x):
        return self.x == x
    
    def __contains__(self, y):
        return y in self.a
    
o = ReloadClass()

if o == 10:
    print("True")
else:
    print("False")
    
if 3 in o:
    print("True")
else:
    print("False")

False
True


In [9]:
'''Декораторы класса'''
def deco(d):
    print("Декоратор")
    return d 

@deco 
class SampleClass:
    def __init__(self, value):
        self.v = value
        
o = SampleClass(1)
print(o.v)

Декоратор
1


In [1]:
'''Вызываем оригинальный метод родительского класса'''
class Person():
    def __init__(self, name):
        self.name = name
        
class EmailPerson(Person):
    def __init__(self, name, email):
        super().__init__(name)
        self.email = email
        
bob = EmailPerson('Bob Frapples', 'bob@frapples.com')
print(bob.name)
print(bob.email)

Bob Frapples
bob@frapples.com


In [5]:
class A():
    count = 0
    def __init__(self):
        A.count += 1
        
    def exclaim(self):
        print("I'm an A!")
        
    @classmethod
    def kids(cls):
        print(f"A has {cls.count} little objects.")
        
easy_a = A()
breezy_a = A()
wheezy_a = A()
A.kids()

A has 3 little objects.


In [13]:
'''Утиная типизация'''
class Quote:
    def __init__(self, person, words):
        self.person = person
        self.words = words
        
    def who(self):
        return self.person
    
    def says(self):
        return self.words + '.'
    
class QuestionQuote(Quote):
    def says(self):
        return self.words + '?'
    
class ExclamationQuote(Quote):
    def says(self):
        return self.words + '!'

   
hunter = Quote('Elmer Fudd', "I'm hunting wabbits")
print(hunter.who(), 'says:', hunter.says())

hunted1 = QuestionQuote('Bugs Bunny', "What's up, doc")
print(hunted1.who(), 'says:', hunted1.says())

hunted2 = ExclamationQuote('Daffy Duck', "It's rabbit season")
print(hunted2.who(), 'says:', hunted2.says())

print('-' * 50)

class BabblingBrook:
    def who(self):
        return 'Brook'
    
    def says(self):
        return 'Babble'

brook = BabblingBrook()

def who_says(obj):
    print(obj.who(), 'says', obj.says())
    
who_says(hunter)    
who_says(hunted1)    
who_says(hunted2)    
who_says(brook)       

Elmer Fudd says: I'm hunting wabbits.
Bugs Bunny says: What's up, doc?
Daffy Duck says: It's rabbit season!
--------------------------------------------------
Elmer Fudd says I'm hunting wabbits.
Bugs Bunny says What's up, doc?
Daffy Duck says It's rabbit season!
Brook says Babble


In [28]:
'''Особые методы'''
class Word:
    def __init__(self, text):
        self.text = text 
        
    def __eq__(self, word2):
        return self.text.lower() == word2.text.lower()
    
    def __str__(self):
        return self.text
    
    def __repr__(self):
        return f"{Word(self.text)}"
    
first = Word('ha')
second = Word('HA')
third = Word('eh')
print(first == second)
print(first == third)

print('-' * 10)

first = Word('ha')
first  # используется __repr__
# print(first)  # используется __str__

True
False
----------


ha

In [30]:
'''Композиция'''
class Bill:
    def __init__(self, description):
        self.description = description
        
class Tail:
    def __init__(self, length):
        self.length = length
        
class Duck:
    def __init__(self, bill, tail):
        self.bill = bill
        self.tail = tail
        
    def about(self):
        print('This duck has a', bill.description, 'bill and a', tail.length, 'tail')
        

tail = Tail('long')
bill = Bill('wide orange')
duck = Duck(bill, tail)
duck.about()

This duck has a wide orange bill and a long tail
